In [5]:
import pandas as pd
import os, sys, importlib
sys.path.append(os.path.abspath(".."))
from utils import images_clf
import json

## TEST : autoclf

## labels2emb

In [ ]:
## labels_text (prompt):
#？
is_default_pic_labels = {
    "1": "the official Airbnb default profile picture, a gray geometric human silhouette",
    "0": "a normal user-uploaded profile picture"
}

has_person_labels = {
    "1": "a photo that contains one or more people",
    "0": "a photo without any people"
}
# type_labels = {
#     "life": "a person shown in a real-life scene or activity, with visible environment or lifestyle context.",
#     "pro": "a clean portrait or headshot focused mainly on the face, with little or no background information.",
#     "UNK": "no visible person, or not enough information to determine lifestyle vs portrait."
# }

type_labels = {
    "life": 
        "a photo of a person in a visible daily scene or some activities",
    "pro": 
        "a portrait or headshot,focused mainly on the face, with little or no background information.",
    "UNK": 
        "an image without any people or cannot determine whether it is lifestyle or portrait"
}
quality_labels = {
    "high": 
        "a clear, high-quality photo with good lighting and sharp details",
    "low": 
        "a low-quality photo with blur, noise, poor lighting or distortion",
    "UNK": 
        "quality cannot be determined"
}
is_smiling_labels = {
    "1": "a person smiling visibly",
    "0": "a person not smiling",
    "UNK": "no person or cannot see their face"
}
sex_labels = {
    "M": "a photo of a man",
    "F": "a photo of a woman",
    "MIX": "a photo with multiple people of mixed gender",
    "UNK": "the gender of the person cannot be determined or no person present"
}
labels_text = {
    "type": type_labels,
    "quality": quality_labels,
    "is_smiling": is_smiling_labels,
    "sex": sex_labels,
    "has_person": has_person_labels,
    "is_default_pic": is_default_pic_labels
}

with open ("labels/labels_text.json","w", encoding='utf-8') as f :
    json.dump(labels_text, f, indent=2)


In [15]:
# embed labels :
import torch
import numpy as np
from transformers import CLIPProcessor, CLIPModel
device='cuda' if torch.cuda.is_available() else "cpu" 
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 12e9ac5f-7195-4abf-94f9-6b48dfe3f542)')' thrown while requesting HEAD https://huggingface.co/openai/clip-vit-base-patch32/resolve/main/config.json
Retrying in 1s [Retry 1/5].


In [16]:

# 1) 读取你的 labels JSON
with open("labels/labels_text.json", "r") as f:
    labels_text = json.load(f)

def embed_text_by_clip(text_list):
    """
    text_list: list of strings
    return: np.array of shape (len(text_list), embedding_dim)
    label 被省去，只留下具体描述

    """
    with torch.no_grad():
        inputs = processor(text=text_list, return_tensors="pt", padding=True).to(device)
        text_features = model.get_text_features(**inputs)  # (N, 512)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        return text_features.cpu().numpy().astype("float32")


# 2) 生成 embedding
labels_emb = {}
for category, dic in labels_text.items():
    texts = list(dic.values())  # e.g. ["life prompt", "pro prompt", "UNK prompt"]
    emb = embed_text_by_clip(texts)  # shape = (num_classes, 512)
    labels_emb[category] = emb


# 3) 保存到单个 npz 文件
np.savez("labels/labels_emb.npz", **labels_emb)
print("[SUCCES] Saved text embeddings → labels/labels_emb.npz")


[SUCCES] Saved text embeddings → labels/labels_emb.npz


In [ ]:
#检查：
emb = np.load("labels/labels_emb.npz")
type_emb = emb["type"]       # (3, 512)
quality_emb = emb["quality"] # (3, 512)
sex_emb = emb["sex"]         # (4, 512)
print(type_emb.shape)
# 假设有一个 image embedding img_emb (1, 512)

# import numpy as np
# pred_idx = np.argmax(np.dot(img_emb, type_emb.T))
# pred_label = list(labels_text["type"].keys())[pred_idx]
# print(pred_label)


(3, 512)


In [ ]:
# prediction：

def zero_shot_predict(label_type, image_emb, text_emb_dict):
    """
    image_emb: shape (512,)
    text_emb_dict: {"life": (512,), "pro": (512,), ...}
    """
    labels = list(text_emb_dict.keys())
    text_embs = np.stack([text_emb_dict[k] for k in labels])  # (K, 512)

    # cosine similarity
    scores = image_emb @ text_embs.T
    best = labels[np.argmax(scores)]
    return best, scores

In [ ]:
# ---------------------------------------------------
# 开始自动分类
# ---------------------------------------------------
pred = {}

for fname, img_emb in emb_dict.items():
    result = {}

    # -----------------------------
    # (A) has_person
    # -----------------------------
    zlabel, score = zshot_predict(img_emb, ["person", "no_person"])
    result["has_person"] = 1 if zlabel == "person" else 0

    # -----------------------------
    # (B) is_default_pic（你的要求）
    # -----------------------------
    sim_default = sim(img_emb, default_emb)
    result["is_default_pic"] = 1 if sim_default > 0.94 else 0

    # -----------------------------
    # (C) type
    # -----------------------------
    if result["has_person"] == 0:
        result["type"] = "UNK"
    else:
        zlabel, _ = zshot_predict(img_emb, ["life", "pro"])
        result["type"] = zlabel

    # -----------------------------
    # (D) is_smiling
    # -----------------------------
    if result["has_person"] == 0:
        result["is_smiling"] = "UNK"
    else:
        zlabel, _ = zshot_predict(img_emb, ["smiling", "not_smiling"])
        result["is_smiling"] = "1" if zlabel == "smiling" else "0"

    # -----------------------------
    # (E) sex
    # -----------------------------
    if result["has_person"] == 0:
        result["sex"] = "UNK"
    else:
        zlabel, _ = zshot_predict(img_emb, ["man", "woman", "mixed"])
        if zlabel == "man":
            result["sex"] = "M"
        elif zlabel == "woman":
            result["sex"] = "F"
        else:
            result["sex"] = "MIX"

    # -----------------------------
    # (F) quality
    # -----------------------------
    if result["has_person"] == 0:
        result["quality"] = "UNK"
    else:
        zlabel, _ = zshot_predict(img_emb, ["high_quality", "low_quality"])
        result["quality"] = "high" if zlabel == "high_quality" else "low"

    pred[fname] = result


# 保存结果
with open("annotations_SAMPLE/autoclf_predictions.json", "w") as f:
    json.dump(pred, f, indent=2)

print("✓ AutoCLF 预测完成！已保存至 autoclf_predictions.json")
